<a href="https://colab.research.google.com/github/MontyRex/ML-labs/blob/main/Test_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')


def connect_to_database(db_path: str = 'Кейс_Аналитика.db'):
    """Подключение к SQLite базе данных и загрузка всех таблиц"""

    try:
        # Создаем соединение с базой данных
        conn = sqlite3.connect(db_path)
        print(f"✅ Успешное подключение к базе данных: {db_path}")

        # Получаем список всех таблиц в базе
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()

        table_names = [table[0] for table in tables]
        print(f"📋 Найдены таблицы: {table_names}")

        return conn, table_names

    except Exception as e:
        print(f"❌ Ошибка подключения к базе данных: {e}")
        return None, []

def load_tables_from_db(conn, table_names: List[str]) -> Dict[str, pd.DataFrame]:
    """Загрузка всех таблиц из базы данных в DataFrame"""

    tables = {}

    for table_name in table_names:
        try:
            # Загружаем таблицу в DataFrame
            query = f"SELECT * FROM {table_name}"
            tables[table_name] = pd.read_sql_query(query, conn)
            print(f"✅ {table_name}: {tables[table_name].shape[0]} строк, {tables[table_name].shape[1]} столбцов")

        except Exception as e:
            print(f"❌ Ошибка загрузки таблицы {table_name}: {e}")

    return tables


# ### 2. АНАЛИЗ СХЕМЫ БАЗЫ ДАННЫХ
def analyze_database_schema(conn):
    """Анализ структуры базы данных и связей между таблицами"""

    print("\n=== АНАЛИЗ СХЕМЫ БАЗЫ ДАННЫХ ===")

    cursor = conn.cursor()

    # Получаем информацию о всех таблицах
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    for table in tables:
        table_name = table[0]
        print(f"\n📊 ТАБЛИЦА: {table_name}")

        # Получаем информацию о колонках
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = cursor.fetchall()

        for col in columns:
            col_id, col_name, col_type, not_null, default_value, pk = col
            pk_flag = " 🔑" if pk > 0 else ""
            print(f"   {col_name} ({col_type}){pk_flag}")


# ### 3. ВЫПОЛНЕНИЕ SQL-ЗАПРОСОВ ДЛЯ ПРОВЕРКИ ДАННЫХ
def run_data_quality_sql(conn):
    """Выполнение SQL-запросов для проверки качества данных"""

    print("\n=== ПРОВЕРКА КАЧЕСТВА ДАННЫХ ЧЕРЕЗ SQL ===")

    quality_queries = {
        "Общее количество отключений": "SELECT COUNT(*) as total_blackouts FROM blackouts",
        "Отключения с некорректными датами": """
            SELECT COUNT(*) as invalid_dates
            FROM blackouts
            WHERE end_date < start_date OR start_date IS NULL
        """,
        "Отключения без привязки к зданиям": """
            SELECT COUNT(DISTINCT b.id) as orphaned_blackouts
            FROM blackouts b
            LEFT JOIN blackouts_buildings bb ON b.id = bb.blackout_id
            WHERE bb.blackout_id IS NULL
        """,
        "Здания без привязки к улицам": """
            SELECT COUNT(*) as buildings_no_street
            FROM buildings b
            LEFT JOIN streets s ON b.street_id = s.id
            WHERE s.id IS NULL
        """,
        "Статистика по типам отключений": """
            SELECT type, COUNT(*) as count,
                   AVG((julianday(end_date) - julianday(start_date)) * 24) as avg_duration_hours
            FROM blackouts
            WHERE end_date IS NOT NULL AND start_date IS NOT NULL
            GROUP BY type
            ORDER BY count DESC
        """
    }

    for query_name, query in quality_queries.items():
        try:
            result = pd.read_sql_query(query, conn)
            print(f"\n📈 {query_name}:")
            print(result.to_string(index=False))
        except Exception as e:
            print(f"❌ Ошибка в запросе '{query_name}': {e}")



# ### 4. СОЗДАНИЕ ЕДИНОГО ДАТАСЕТА ЧЕРЕЗ SQL JOIN
def create_combined_dataset_sql(conn):
    """Создание объединенного датасета через SQL запросы"""

    print("\n=== СОЗДАНИЕ ОБЪЕДИНЕННОГО ДАТАСЕТА ===")

    # Основной запрос для объединения всех таблиц
    main_query = """
    SELECT
        b.*,
        bb.building_id,
        build.street_id as building_street_id,
        build.number as building_number,
        build.district_id as building_district_id,
        build.is_fake as building_is_fake,
        build.folk_district_id as building_folk_district_id,
        build.big_folk_district_id as building_big_folk_district_id,
        build.type as building_type,
        build.city_id as building_city_id,
        build.coordinates as building_coordinates,
        s.name as street_name,
        s.city_id as street_city_id,
        d.name as district_name,
        fd.name as folk_district_name,
        bfd.name as big_folk_district_name,
        c.name as city_name

    FROM blackouts b
    LEFT JOIN blackouts_buildings bb ON b.id = bb.blackout_id
    LEFT JOIN buildings build ON bb.building_id = build.id
    LEFT JOIN streets s ON build.street_id = s.id
    LEFT JOIN districts d ON build.district_id = d.id
    LEFT JOIN folk_districts fd ON build.folk_district_id = fd.id
    LEFT JOIN big_folk_districts bfd ON build.big_folk_district_id = bfd.id
    LEFT JOIN cities c ON build.city_id = c.id
    """

    try:
        # Загружаем объединенные данные
        combined_data = pd.read_sql_query(main_query, conn)
        print(f"✅ Объединенный датасет создан: {combined_data.shape[0]} строк, {combined_data.shape[1]} столбцов")

        return combined_data

    except Exception as e:
        print(f"❌ Ошибка создания объединенного датасета: {e}")
        return None



# ### 5. БАЗОВЫЙ УРОВЕНЬ - ПОЛНАЯ ПОДГОТОВКА ЧЕРЕЗ SQLite
def comprehensive_data_preparation_db(db_path: str = 'Кейс_Аналитика.db'):
    """Полный пайплайн подготовки данных через SQLite"""

    print("🚀 ЗАПУСК ПОЛНОЙ ПОДГОТОВКИ ИЗ SQLite БАЗЫ")

    # 1. Подключаемся к базе
    conn, table_names = connect_to_database(db_path)
    if conn is None:
        return None, None

    # 2. Анализируем схему
    analyze_database_schema(conn)

    # 3. Проверяем качество данных через SQL
    run_data_quality_sql(conn)

    # 4. Создаем объединенный датасет
    combined_data = create_combined_dataset_sql(conn)

    if combined_data is None:
        conn.close()
        return None, None

    # 5. Загружаем отдельные таблицы для справки
    tables = load_tables_from_db(conn, table_names)

    # 6. Закрываем соединение
    conn.close()
    print("✅ Соединение с базой данных закрыто")

    return combined_data, tables



# ### 6. ПРЕОБРАЗОВАНИЕ ДАННЫХ И СОЗДАНИЕ ПРИЗНАКОВ
def process_combined_data(combined_data: pd.DataFrame):
    """Обработка объединенных данных и создание признаков"""

    print("\n=== ОБРАБОТКА ДАННЫХ И СОЗДАНИЕ ПРИЗНАКОВ ===")

    df = combined_data.copy()

    # Преобразование дат
    date_columns = ['start_date', 'end_date']
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    # Создание признаков
    if 'start_date' in df.columns:
        # Длительность отключения
        df['duration_hours'] = (df['end_date'] - df['start_date']).dt.total_seconds() / 3600

        # Временные признаки
        df['start_hour'] = df['start_date'].dt.hour
        df['day_of_week'] = df['start_date'].dt.day_name()
        df['day_of_week_num'] = df['start_date'].dt.dayofweek
        df['is_weekend'] = df['start_date'].dt.dayofweek >= 5
        df['month'] = df['start_date'].dt.month
        df['year'] = df['start_date'].dt.year
        df['season'] = (df['start_date'].dt.month % 12 + 3) // 3

        # Категории времени суток
        df['time_of_day'] = pd.cut(df['start_hour'],
                                  bins=[0, 6, 12, 18, 24],
                                  labels=['Ночь', 'Утро', 'День', 'Вечер'],
                                  right=False)

        # Категории длительности
        df['duration_category'] = pd.cut(df['duration_hours'],
                                       bins=[0, 1, 6, 24, 168, float('inf')],
                                       labels=['<1ч', '1-6ч', '6-24ч', '1-7д', '>7д'])

    # Очистка текстовых полей
    text_columns = ['type', 'initiator_name', 'source', 'description']
    for col in text_columns:
        if col in df.columns:
            df[col] = df[col].fillna('Не указано').str.strip()

    print(f"✅ Обработка завершена: {df.shape[0]} строк, {df.shape[1]} столбцов")
    return df



# ### ОСНОВНОЙ ПАЙПЛАЙН ДЛЯ SQLite
def main_sqlite_pipeline(db_path: str = 'Кейс_Аналитика.db'):
    """Основной пайплайн для работы с SQLite базой"""

    # Запускаем подготовку данных
    combined_data, tables = comprehensive_data_preparation_db(db_path)

    if combined_data is None:
        print("❌ Не удалось подготовить данные")
        return None, None

    # Обрабатываем данные
    processed_data = process_combined_data(combined_data)

    print(f"\n🎉 ПОДГОТОВКА ИЗ SQLite ЗАВЕРШЕНА!")
    print(f"📊 Итоговый датасет: {processed_data.shape[0]:,} строк, {processed_data.shape[1]} столбцов")

    if 'start_date' in processed_data.columns:
        print(f"📅 Диапазон дат: {processed_data['start_date'].min()} - {processed_data['start_date'].max()}")

    # Сохраняем результат
    processed_data.to_csv('blackouts_processed_from_db.csv', index=False, encoding='utf-8')
    print("💾 Данные сохранены в blackouts_processed_from_db.csv")

    return processed_data, tables



# ### БЫСТРЫЙ СТАРТ - ПРОСТАЯ ПРОВЕРКА
def quick_database_check():
    """Быстрая проверка подключения к базе данных"""

    try:
        conn = sqlite3.connect('Кейс_Аналитика.db')

        print("✅ Успешное подключение к Кейс_Аналитика.db")

        # Показываем все таблицы
        tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)
        print(f"📋 Таблицы в базе: {list(tables['name'])}")

        # Быстрая проверка основных таблиц
        for table in ['blackouts', 'buildings', 'blackouts_buildings']:
            if table in tables['name'].values:
                count = pd.read_sql_query(f"SELECT COUNT(*) as count FROM {table}", conn).iloc[0]['count']
                print(f"   {table}: {count:,} записей")
            else:
                print(f"   {table}: ❌ не найдена")

        conn.close()
        return True

    except Exception as e:
        print(f"❌ Ошибка подключения: {e}")
        return False

# Быстрая проверка
quick_database_check()
# ### ЗАПУСК ПАЙПЛАЙНА
print("🔍 Подключаемся к базе данных Кейс_Аналитика.db...")
final_data, raw_tables = main_sqlite_pipeline('Кейс_Аналитика.db')

✅ Успешное подключение к Кейс_Аналитика.db
📋 Таблицы в базе: ['cities', 'districts', 'folk_districts', 'big_folk_districts', 'streets', 'buildings', 'blackouts', 'blackouts_buildings']
   blackouts: 25,264 записей
   buildings: 58,199 записей
   blackouts_buildings: 1,419,556 записей
🔍 Подключаемся к базе данных Кейс_Аналитика.db...
🚀 ЗАПУСК ПОЛНОЙ ПОДГОТОВКИ ИЗ SQLite БАЗЫ
✅ Успешное подключение к базе данных: Кейс_Аналитика.db
📋 Найдены таблицы: ['cities', 'districts', 'folk_districts', 'big_folk_districts', 'streets', 'buildings', 'blackouts', 'blackouts_buildings']

=== АНАЛИЗ СХЕМЫ БАЗЫ ДАННЫХ ===

📊 ТАБЛИЦА: cities
   id (TEXT) 🔑
   name (TEXT)

📊 ТАБЛИЦА: districts
   id (TEXT) 🔑
   name (TEXT)

📊 ТАБЛИЦА: folk_districts
   id (TEXT) 🔑
   name (TEXT)

📊 ТАБЛИЦА: big_folk_districts
   id (TEXT) 🔑
   name (TEXT)

📊 ТАБЛИЦА: streets
   id (TEXT) 🔑
   name (TEXT)
   city_id (TEXT)

📊 ТАБЛИЦА: buildings
   id (TEXT) 🔑
   street_id (TEXT)
   number (TEXT)
   district_id (TEXT)
   is_f